In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.compose import make_column_transformer
import copy
from pandas import Series
import netaddr, time, sys, json, zipfile, csv, geopandas, re, os
import pandas as pd
from zipfile import ZipFile
from io import TextIOWrapper, BytesIO
from collections import defaultdict
from matplotlib import pyplot as plt

In [2]:
train_users = pd.read_csv("data/train_users.csv")
train_logs = pd.read_csv("data/train_logs.csv")
train_y = pd.read_csv("data/train_y.csv")
test_users = pd.read_csv("data/test1_users.csv")
test_logs = pd.read_csv("data/test1_logs.csv")
test_y = pd.read_csv("data/test1_y.csv")
ips = pd.read_csv("data/ip2location.csv")

In [9]:
class UserPredictor():
    def fit(self, users, logs, ys):
        self.users = self.total_min_ips(users, logs)
        self.users = self.ip_to_country(self.users)
        self.merged = pd.merge(left=self.users, right=ys, left_on="id", right_on="id")
        self.pipeline = Pipeline([
    ("both", make_column_transformer((OneHotEncoder(), ["badge", "age", "country"]), 
                                     (PolynomialFeatures(degree=1, include_bias=False), ["past_purchase_amt", "total_min", "avg_minutes", "num_visited"]), remainder="passthrough")),
    ("lr", LogisticRegression(max_iter=1025))
])
        self.pipeline.fit(self.merged[["badge", "age", "country", "past_purchase_amt", "total_min", "avg_minutes", "num_visited"]], self.merged["y"])
    
    def predict(self, users, logs):
        self.users = self.total_min_ips(users, logs)
        self.users = self.ip_to_country(self.users)
        self.users["predicted"] = self.pipeline.predict(self.users[["badge", "age", "country", "past_purchase_amt", "total_min", "avg_minutes", "num_visited"]])
        return self.users["predicted"].values
    
    def total_min_ips(self, users, logs):
        avg = []
        num = []
        total = []
        net = []
        ids = list(users["id"].values)
        for id_ in ids: 
            test = logs[logs["id"] == id_]
            total.append(test["minutes_on_page"].sum())
            avg.append(test["minutes_on_page"].mean())
            num.append(len(test))
            try:
                net.append(int(netaddr.IPAddress(test["ip_address"].values[-1])))
            except:
                net.append(0)
        users["avg_minutes"] = avg
        users["total_min"] = total
        users["num_visited"] = num
        users["netaddr"] = net
        return users.fillna(0)
        
    
    def ip_to_country(self, df):
        df = df.sort_values("netaddr")
        countries = list()
        ip_idx = 0
        for ip in df["netaddr"]:
            for low in ips["low"][ip_idx:]:
                if low <= ip <= ips["high"][ip_idx]:
                    countries.append(ips["region"][ip_idx])
                    break
                else:
                    ip_idx += 1
        df["country"] = countries
        return df.sort_values("id")
    

In [10]:
model = UserPredictor()
train_users = pd.read_csv("data/train_users.csv")
train_logs = pd.read_csv("data/train_logs.csv")
train_y = pd.read_csv("data/train_y.csv")

In [11]:
model.fit(train_users, train_logs, train_y)

In [12]:
test_users = pd.read_csv("data/test1_users.csv")
test_logs = pd.read_csv("data/test1_logs.csv")

In [13]:
y_pred = model.predict(test_users, test_logs)
y_pred

array([0, 0, 1, ..., 0, 1, 1])